In [1]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

from gensim.models import KeyedVectors

import json
import re
import string

In [2]:
# Opening JSON file
f = open('MMHS150K_GT.json')
 
# returns JSON object as 
# a dictionary
data = json.load(f)
texts = []
labels = []
# Closing file
f.close()
# Iterating through the json
# list
for i in data:
    text = data[i]["tweet_text"].split("https://")[0]
    text = text.lower()
    mentions = r'@[^ ]'
    text = re.sub(mentions, '', text)
    text = text.translate(str.maketrans('','',string.punctuation))
    texts.append(text)

    label = data[i]["labels_str"][0]
    labels.append(label)

In [3]:
# Load GLOVE model
glove_model_path = "glove.6B.100d.txt"  # Replace with the path to your GLOVE file
glove_model = KeyedVectors.load_word2vec_format(glove_model_path, binary=False, no_header=True)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(texts, labels, test_size=0.2, random_state=42)
print("split done")
# Create document vectors using average GLOVE word vectors
def get_doc_vector(doc, model):
    words = doc.split()
    vectors = [model[word] for word in words if word in model]
    if not vectors:
        return np.zeros(model.vector_size)
    return np.mean(vectors, axis=0)

X_train_vectors = np.array([get_doc_vector(doc, glove_model) for doc in X_train])
X_test_vectors = np.array([get_doc_vector(doc, glove_model) for doc in X_test])
print("training model")
# Train a support vector machine (SVM) classifier
clf = SVC(kernel='linear')
clf.fit(X_train_vectors, y_train)
print("model trained")
# Make predictions on the test set
y_pred = clf.predict(X_test_vectors)

split done
training model
model trained


In [4]:
# Evaluate the classifier
accuracy = accuracy_score(y_test, y_pred)
print(f"Testing Accuracy: {accuracy:.8f}")

Testing Accuracy: 0.70028366
